___
# Ciência dos Dados - PROJETO 3 - INSPER

___
## Leonardo Bhering 

## Gustavo Molina

___

## INTRODUÇÃO

Este projeto tem como objetivo prever qual o valor de mercado de um jogador de futebol com base em algumas variaveis, como sua idade e seus números da temporada.

___
## MINERANDO DADOS e CARACTERÍSTICAS DO DATASET

[Detalhe aqui as características da base de dados além da análise descritiva.]

In [1]:
import pandas as pd
import os

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Leonardo\Documents\INSPER\CiDadosDP\Projetos\Projeto 3


### BASE DE DADOS

In [9]:
dados = pd.read_excel('jogadores.xlsx')

In [10]:
dados.dtypes

Rank                                       int64
Jogador                                   object
Posição                                   object
Idade                                      int64
Valor de Mercado (em milhões de euros)     int64
Jogos                                      int64
Gols                                       int64
Assistencias                               int64
dtype: object

In [11]:
dados.head(15)

,Rank,Jogador,Posição,Idade,Valor de Mercado (em milhões de euros),Jogos,Gols,Assistencias
0,1,Neymar,Extremo Esquerdo,26,180,32,29,19
1,2,Lionel Messi,Extremo Direito,30,180,55,45,18
2,3,Kylian Mbappé,Extremo Direito,19,120,50,23,19
3,4,Harry Kane,Ponta de Lança,24,120,48,41,5
4,5,Cristiano Ronaldo,Ponta de Lança,33,120,45,46,8
5,6,Kevin De Bruyne,Médio Ofensivo,26,110,55,13,23
6,7,Paulo Dybala,Segundo Avançado,24,100,48,26,7
7,8,Antoine Griezmann,Ponta de Lança,27,100,53,30,15
8,9,Philippe Coutinho,Extremo Esquerdo,25,100,45,23,14
9,10,Eden Hazard,Extremo Esquerdo,27,100,54,18,14


### ANÁLISE DESCRITIVA

___
## MODELOS DE PREDIÇÃO

[O tópico C.PROCESSO E ESTATÍSTICAS DE VALIDAÇÃO estará dentro deste tópico. Organize-o aqui de forma clara!]

[ORGANIZE AQUI TODOS OS MODELOS DE PREDIÇÃO NECESSÁRIOS NESTE PROJETO!]

o MODELO DE PREDIÇÃO PELA MÉDIA

o MODELO DOS K VIZINHOS MAIS PRÓXIMOS (K-Nearest Neighbors Regression)

o MODELO DE REGRESSÃO LINEAR (Multiple Linear Regression)

o MODELO DE ÁRVORES DE REGRESSÃO (Decision Tree Regression)

___
## CONCLUSÃO

___
## REFERÊNCIAS BIBLIOGRÁFICAS